In [ ]:
!pip install python-igraph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Benchmark network 

In [ ]:
#read benchmark graph
#save as gml file
G = nx.read_edgelist('/content/drive/MyDrive/Data/sourcefile/network_100k_0.00-mu_1.pairs', create_using=nx.Graph(), nodetype = int)
print (nx.info(G))
nx.write_gml(G, "/content/drive/MyDrive/Data/sourcefile/test.gml")

In [ ]:
# function for checking the communities
# input should be community file from benchmark

def makeDict(filename) :
    file = open(filename, "r")
    
    comms = []
    nodes = []
    
    for line in file : 
        node = line.split()
        nodes.append(node[0])
        comms.append(int(node[1])-1)
        
    return nodes, comms

nodes , comms = makeDict("/content/drive/MyDrive/Data/sourcefile/community_50k_0.00-mu_1.txt")
print("the number of communities in this network is {}".format(len(set(comms))))

In [ ]:
# get ground truth modularity of benchmark network
partition = ig.VertexClustering(G, membership=list(comms))
optim_modul = partition.q

In [ ]:
#!/usr/bin/env python
'''
Module to handle constrained simulated annealing calculations for community detection in complex networks.
'''

# ================================= IMPORTS ==============================
# Built-ins
import os
import sys
import pickle
import random

# debugging
import time

# External
import igraph as ig
import numpy as np
import copy
import pandas as pd

# Custom

# ----------------------------
__author__ = ["Daniel Kaiser", "Sangpil Youm"]
__credits__ = ["Daniel Kaiser", "Sangpil Youm"]

__version__ = "0.4"
__maintainer__ = "Daniel Kaiser"
__email__ = "kaiserd@iu.edu"
__status__ = "Development"


# ====================== CLASSES AND FUNCTIONS =======================
class Annealing:
    '''Class to handle the network data to perform constrained SA on'''

    def __init__(self, ntwk, number_of_groups, temp, cooling):
        '''Constructor for SANetwork class
        Inputs :
            ntwk : networkx Network
                ntwk is a networkx Network class objct
            number_of_groups : positive int
                The number of communities to find
        Returns :
            None
        '''
        self.ntwk = ntwk
        self.n = ntwk.vcount()

        self.l_modul_list = []
        self.g_modul_list = []

        self.t_modularity_list = []
        self.modularity = 0
        self.temp = temp
        self.cooling = cooling
        self.f_comm = {}

        self.number_of_groups = number_of_groups

        # initial grouping with constrained number of groups
        self.comm = {node.index: np.random.randint(0, number_of_groups) for node in list(ntwk.vs())}  # number of groups
        self.comms_set = list(set(self.comm.values()))

        num_comm = len(self.comms_set)

        while num_comm != self.number_of_groups :
          self.comm = {node.index: np.random.randint(0, number_of_groups) for node in list(ntwk.vs())}  
          self.comms_set = list(set(self.comm.values()))
          num_comm = len(self.comms_set)


        

    def GlobalMove(self, elligible):

        comm = self.comm
        # this is number of community
        # there is no empty group
        num_comm = 0 

        while num_comm != self.number_of_groups :
          
          node_choice = [i for i in range(2, len(elligible)+1)] #list of number for nodes being passed in 
          node_num = np.random.choice(node_choice)
          node_list = list(set(np.random.choice(elligible, node_num, replace=False))) #Take several random nodes (2~ length of elligible)

          s_node = np.random.choice(node_list) #select single node to check value
          selected = comm[s_node]
          new_comms_list = [val for val in self.comms_set if val != selected]

          #change values in all processing data
          change_comm = np.random.choice(new_comms_list) # number for new comm
          for node in node_list :
            comm[node] = change_comm
          
          num_comm = len(list(set(comm.values())))
          

        partition = ig.VertexClustering(self.ntwk, membership= list(comm.values()))
        gtemp_modularity = partition.q

        #print("problem", gtemp_modularity)
        better = bool(gtemp_modularity >= self.modularity)
        temp_move = bool(np.random.rand() <= np.exp((gtemp_modularity - self.modularity) * (1/self.temp)))

        if better or temp_move :

          self.comm = comm
          self.modularity = gtemp_modularity
          self.t_modularity_list.append(gtemp_modularity)
          self.temp *= self.cooling

          return True
        
        else:
          self.temp *= self.cooling

          return False

    def helper_GlobalMoves(self, g_elligible, num_moves=100,comm = {}, comm_choice=0, stop=5):
        num_moves = self.n
        comm = self.comm

        elligible = g_elligible
        g_len_elli = len(elligible) 
        
        while g_len_elli < 2 : #length of elligible for global movements at least 2
          comm_choice = np.random.choice(list(self.comms_set))
          elligible = [node for node, comm in self.comm.items() if comm == comm_choice]
          g_len_elli = len(elligible)

        for _ in range(num_moves) :
          val = self.GlobalMove(elligible)
          
          if val:  # check if need to consider the other community now

              comm_choice = np.random.choice(list(self.comms_set))
              self.g_modul_list.append(self.modularity)
              elligible = [node for node, comm in self.comm.items() if comm == comm_choice]  # redesignate which nodes are elligible

              g_len_elli = len(elligible)      
              while g_len_elli < 2 :
                  #print("AHHHHHHHHHHH")
                  comm_choice = np.random.choice(list(self.comms_set))
                  elligible = [node for node, comm in self.comm.items() if comm == comm_choice]
                  g_len_elli = len(elligible)

          else:
                continue

          
          f_comm = copy.deepcopy(comm)
          self.f_comm = f_comm
          elligible = copy.deepcopy(elligible)
          self.g_modul_list = copy.deepcopy(self.g_modul_list)
          print("g",self.g_modul_list[-1])
  

        #print("\n\n\n ~~~~~~~~~~~~~~~~~~ \n Final modularity is last {} \n community {} \n ~~~~~~~~~~~~~~~~~".format(modul_list[-1],final_comm))
        return self.f_comm, elligible                
          

    def LocalMove(self,elligible):
        # Make a copy to avoid prematurely altering "true" communities
        comm = self.comm

        # this is number of community
        # there is no empty group
        num_comm = 0
 
        while num_comm != self.number_of_groups :
          node = np.random.choice(elligible)  # Take a random node from the community being passed in
          
          # Making the local move
          new_comms = set(comm.values())

          # two cases one for all nodes has same comm, and others not
          # fix here by sangpil
          #if len(new_comms) == 1:
          #    selected = comm[node]
              # get from ground truth set of comms
          #    new_comms_list = [val for val in self.comms_set if val != selected]
          #    comm[node] = np.random.choice(new_comms_list)
          #else:
          selected = comm[node]
          new_comms_list = [val for val in self.comms_set if val != selected]
              #print(new_comms_list)
          comm[node] = np.random.choice(new_comms_list)
          
          num_comm = len(list(set(comm.values())))
          #print(num_comm, "stuck here2")

        # Getting modularity of post-local move partitions
        partition = ig.VertexClustering(self.ntwk, membership=list(comm.values()))
        ltemp_modularity = partition.q

        # If move is better, adjust the community and modularity accordingly
        # Alternatively, if move is worse but succeed temperature calculations
        better = bool(ltemp_modularity >= self.modularity)
        temp_move = bool(np.random.rand() <= np.exp((ltemp_modularity - self.modularity) * (1 / self.temp)))

        if better or temp_move:
            self.comm = comm
            self.modularity = ltemp_modularity
            self.t_modularity_list.append(ltemp_modularity)
            self.temp *= self.cooling

            return True
        else:
            self.temp *= self.cooling
            return False

    def helper_LocalMoves(self,elligible, num_moves=100,comm = {}, comm_choice=0, stop=5):
        num_moves = (self.n) **2
        comm = self.comm

        # the set of nodes under a given community
        
        for _ in range(num_moves):
            val = self.LocalMove(elligible)
            #modularity_list_length = len(self.modularity_list)
            if val:  # check if need to consider the other community now

              
                comm_choice = np.random.choice(list(self.comms_set))
                self.l_modul_list.append(self.modularity)
                elligible = [node for node, comm in self.comm.items() if comm == comm_choice]  # redesignate which nodes are elligible
                    
                while not elligible:
                    #print("AHHHHHHHHHHH")
                    comm_choice = np.random.choice(list(self.comms_set))
                    elligible = [node for node, comm in self.comm.items() if comm == comm_choice]
                #print("stuck here")
            else:
                  continue
            
           
            f_comm = copy.deepcopy(comm)
            self.f_comm = f_comm
            elligible = copy.deepcopy(elligible)
            self.l_modul_list = copy.deepcopy(self.l_modul_list)
            
            print("l",self.l_modul_list[-1])
  
        #print(modul_list)
        #print("\n\n\n ~~~~~~~~~~~~~~~~~~ \n Final modularity is last {} \n community {} \n ~~~~~~~~~~~~~~~~~".format(modul_list[-1],final_comm))
        return self.f_comm, elligible

    def total_moves(self, stop, comm_choice =0) :

      start = time.time()
      modularity_repeat = True
      
      elligible = [node for node, comm in self.comm.items() if comm == comm_choice]

      modularity_list = []
      final_comm ={}

      while modularity_repeat == True :

        prob = np.random.rand()
        
        #check the probability for global movea and local moves
        if prob  > 0.9 :
          local_comm, new_l_elligible = self.helper_LocalMoves(elligible)
          elligible = new_l_elligible

        else :
          global_comm , new_g_elligible = self.helper_GlobalMoves(elligible)
          elligible = new_g_elligible


        modularity_list_length = len(self.t_modularity_list)

        #print(self.modularity)
        if modularity_list_length >= stop :
            modularity_repeat = bool(len(set(self.t_modularity_list[-stop:])) != 1)
      
      end = time.time()
      
      print("\n ~~~~~~~~~~~~~~~~~~ \n Final modularity is last {} \n community {} \n  time    {} \n~~~~~~~~~~~~~~~~~".format(self.t_modularity_list[-1],self.f_comm, end-start))
      return self.t_modularity_list[-1], end-start




# Main function 

In [ ]:
#G = ig.read('/content/drive/MyDrive/Data/sourcefile/dolphins.gml')
G = ig.read('/content/drive/MyDrive/Data/sourcefile/50k_1.gml')

mod_list =[]
time_list =[]
cooling_list = []
stop_list = []

#[0.995,0.98,0.97,0.95,0.90]

for cool in [0.999,0.995] :
  for stop in [200,300,400] :
    for _ in range(30) :
      print("Cooling  {},  Stop {}".format(cool, stop))
      Ann = Annealing(G, 28, temp = 99.9, cooling = cool)
      mod, tim = Ann.total_moves(stop = stop)
      
      mod_list.append(mod)
      time_list.append(tim)
      cooling_list.append(cool)
      stop_list.append(stop)

# Saving the result

In [ ]:
import pandas as pd

l_mod =mod_list
l_time = time_list
l_cooling = cooling_list
l_stop = stop_list

df = pd.DataFrame({
        "modularity" : l_mod,
        "time" : l_time,
        "stop" : l_stop,
        "cooling" :l_cooling
        })
df

,modularity,time,stop,cooling
0,0.447411,86.319225,300,0.999
1,0.517127,275.196578,300,0.999
2,0.489696,97.834783,300,0.999
3,0.525869,120.954716,300,0.999
4,0.483367,126.759220,300,0.999
...,...,...,...,...
195,0.423342,108.189181,600,0.980
196,0.491733,139.753890,600,0.980
197,0.482912,106.315088,600,0.980
198,0.483466,156.002537,600,0.980


In [ ]:
df.to_csv('result_1.csv', index = False)

In [ ]:
df.to_csv('/content/drive/MyDrive/Data/Return/result_dolphin_1124.csv', index = False)